In [ ]:
from datascience import *
%matplotlib inline
path_data = '../../../assets/data/'
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=np.VisibleDeprecationWarning)

# Lecture 23 #

## Percentiles ##

In [ ]:
# Manually compute the 55th percentile.
x = make_array(43, 20, 51, 7, 28, 34)

In [ ]:
# Step 1. Sort the data
np.sort(x)

In [ ]:
# Step 2. Figure out where 55th percentile would be.

In [ ]:
np.arange(1, 7)/6

In [ ]:
np.sort(x).item(3)

In [ ]:
# Alternatively: One line of code
percentile(55, x)

In [ ]:
# Discussion question
s = make_array(1, 3, 5, 7, 9)

In [ ]:
percentile(10, s) == 0

In [ ]:
percentile(39, s) == percentile(40, s)

In [ ]:
percentile(40, s) == percentile(41, s)

In [ ]:
percentile(50, s) == 5

## Total Compensation in Population 

In [ ]:
sf = Table.read_table('san_francisco_2019.csv')
sf.show(3)

In [ ]:
# Who made the most money
sf.sort('Total Compensation', descending=True).show(5)

In [ ]:
# Who made the least money
sf.sort('Total Compensation', descending=False).show(5)

In [ ]:
# $15/hr, 20 hr/wk, 50 weeks

min_salary = 15 * 20 * 50
sf = sf.where('Salary', are.above(min_salary))

In [ ]:
sf.num_rows

In [ ]:
sf_bins = np.arange(0, 726000, 25000)
sf.hist('Total Compensation', bins=sf_bins)

## Parameter: Median Total Compensation 

In [ ]:
pop_median = percentile(50, sf.column('Total Compensation'))
pop_median

## Estimating the Parameter (Pretend it is Unknown) 

In [ ]:
our_sample = sf.sample(400, with_replacement=False)

In [ ]:
our_sample.hist('Total Compensation', bins=sf_bins)

In [ ]:
percentile(50, our_sample.column('Total Compensation') )

But in the real world we won't be able to keep going back to the population. How to generate a new random sample *without going back to the population?*

# Bootstrap

Sample randomly
 - from the original sample
 - with replacement
 - the same number of times as the original sample size

In [ ]:
# Default behavior of tbl.sample:
# at random with replacement,
# the same number of times as rows of tbl

bootstrap_sample = our_sample.sample()

In [ ]:
bootstrap_sample.hist('Total Compensation', bins=sf_bins)

## Bootstrap Sample Median
This is one estimate of the population median.

In [ ]:
percentile(50, bootstrap_sample.column('Total Compensation'))

In [ ]:
def one_bootstrap_median():
    # draw the bootstrap sample
    resample = our_sample.sample()
    # return the median total compensation in the bootstrap sample
    return percentile(50, resample.column('Total Compensation'))

In [ ]:
one_bootstrap_median()

In [ ]:
# Generate the medians of 1000 bootstrap samples
num_repetitions = 1000
bstrap_medians = make_array()
for i in np.arange(num_repetitions):
    bstrap_medians = np.append(bstrap_medians, one_bootstrap_median())

In [ ]:
resampled_medians = Table().with_column('Bootstrap Sample Median', bstrap_medians)
median_bins=np.arange(120000, 160000, 2000)
resampled_medians.hist(bins = median_bins)

# Plotting parameters; you can ignore this code
parameter_green = '#32CD32'
plots.ylim(-0.000005, 0.00014)
plots.scatter(pop_median, 0, color=parameter_green, s=40, zorder=2)
plots.title('Bootstrap Medians and the Parameter (Green Dot)');

## Percentile Method: Middle 95% of the Bootstrap Estimates 

In [ ]:
left = percentile(2.5, bstrap_medians)
right = percentile(97.5, bstrap_medians)

make_array(left, right)

In [ ]:
resampled_medians.hist(bins = median_bins)

# Plotting parameters; you can ignore this code
plots.ylim(-0.000005, 0.00014)
plots.plot(make_array(left, right), make_array(0, 0), color='yellow', lw=3, zorder=1)
plots.scatter(pop_median, 0, color=parameter_green, s=40, zorder=2);